# Opening the box of gradient boosting decision trees (GBDT)

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

The aim of this notebook is 2-folds:

- dissecting a pure Python (unoptimized) implementation of GBRT
- use a profiler (here snakeviz) to identify the computational bottleneck.

We will explore the code in the file `tinygbt.py`.

TinyGBT (Tiny Gradient Boosted Trees) is a 200 line gradient boosted trees implementation written in pure Python.

First let's load some regression data.  
Gradient Boosting Decision Trees（GBDT，梯度提升决策树）是一类把很多“弱决策树”按顺序叠加起来、通过梯度下降思想不断修正错误的强模型。
它在结构上是树模型，在思想上是优化问题。单棵树预测能力有限，但很多棵加起来很强。  

我们分类的目的是:minimise loss
每一个样本都有一个weight，$(x_i, y_i,w_i)$. 每次训练新的classifier的时候，改变weight-->  
        错误分类的，基于更大的weight，正确分类的给予更小的weight.每一个literation都进行这个操作，直到正确分类。  
这样实现weak decision --> fort decision


In [1]:
import pandas as pd

print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('datasets/regression.train', header=None, sep='\t')
df_test = pd.read_csv('datasets/regression.test', header=None, sep='\t')

df = pd.concat([df_train, df_test], axis=0)
y = df[0].values
X = df.drop(0, axis=1).values

Loading data...


Now we will split the data in train and test. Here the test data will be
used as **validation set** to do **"early stopping"** i.e. to stop adding trees
to the model if the validation loss increases for multiple successive boosting rounds.

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

#### Now let's run our TinyGBT code

In [7]:
from sklearn.metrics import mean_squared_error
from tinygbt import GBT

print('Start training...')
gbt = GBT(n_estimators=10, gamma=0., lambd=1,
          min_split_gain=0.1, max_depth=5,
          learning_rate=0.3)
gbt.fit(X_train, y_train, valid_set=(X_test, y_test), early_stopping_rounds=5)

print('Start predicting...')
y_pred = gbt.predict(X_test)
print('The MSE of prediction is:', mean_squared_error(y_test, y_pred))

Start training...
Training until validation scores don't improve for 5 rounds.
Iter   0, Train's L2: 0.3586431862, Valid's L2: 0.3570793975, Elapsed: 5.87 secs
Iter   1, Train's L2: 0.1879371547, Valid's L2: 0.2210272602, Elapsed: 5.94 secs
Iter   2, Train's L2: 0.1789110700, Valid's L2: 0.2200622122, Elapsed: 5.94 secs
Iter   3, Train's L2: 0.1763577214, Valid's L2: 0.2190739491, Elapsed: 6.00 secs
Iter   4, Train's L2: 0.1759371507, Valid's L2: 0.2190762446, Elapsed: 5.99 secs
Iter   5, Train's L2: 0.1757874497, Valid's L2: 0.2190423557, Elapsed: 6.36 secs
Iter   6, Train's L2: 0.1757632946, Valid's L2: 0.2190426775, Elapsed: 5.96 secs
Iter   7, Train's L2: 0.1757561206, Valid's L2: 0.2190428308, Elapsed: 6.03 secs
Iter   8, Train's L2: 0.1757539749, Valid's L2: 0.2190428819, Elapsed: 6.17 secs
Iter   9, Train's L2: 0.1757533318, Valid's L2: 0.2190428976, Elapsed: 8.07 secs
Training finished. Elapsed: 62.33 secs
Start predicting...
The MSE of prediction is: 0.21904235565991395


In [ ]:
#pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ---------------------------- ----------- 1.0/1.5 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.5 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


Let's first make sure this code gives comparable results with [lightgbm](https://lightgbm.readthedocs.io/en/latest/)

In [8]:
import lightgbm as lgb

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 1.,
    'bagging_fraction': 1.,
    'bagging_freq': 1,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
rmse_test = mean_squared_error(y_test, y_pred)
print(f'The MSE of prediction is: {rmse_test}')

Starting training...


TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'

And also with scikit-learn [HistGradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html)

In [9]:
# from sklearn.experimental import enable_hist_gradient_boosting  # uncomment this for sklearn < 1.0
from sklearn.ensemble import HistGradientBoostingRegressor
est = HistGradientBoostingRegressor(
    max_iter=10,
    validation_fraction=0.2,
    random_state=42,
    l2_regularization=0.,
    min_samples_leaf=20,
    learning_rate=0.3,
    n_iter_no_change=5).fit(X_train, y_train)
y_pred = est.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred)
print(f'The MSE of prediction is: {rmse_test}')

The MSE of prediction is: 0.19818811241253062


### Let's do a round of profiling using snakeviz

While lightgbm is implemented in C++ and HistGradientBoostingRegressor with Cython, our TinyGBT is implemented in pure Python. The consequence is that our code is much slower. To identify the computational bottleneck let's profile our code. For this we will use snakeviz.SnakeViz 是一个 用于可视化 Python 程序性能分析（profiling）结果的工具。

There are many other profilers such as:
- line_profiler: https://github.com/pyutils/line_profiler (easy line by line profiling for Python code)
- viztracer: https://viztracer.readthedocs.io (allows to profile low level code including compiled Cython code)

In [13]:
pip install snakeviz

Note: you may need to restart the kernel to use updated packages.


In [14]:
%load_ext snakeviz

gbt = GBT(n_estimators=2,  # do only two rounds of boosting
          gamma=0., lambd=1,
          min_split_gain=0.1, max_depth=5,
          learning_rate=0.3)

%snakeviz gbt.fit(X_train, y_train, valid_set=(X_test, y_test), early_stopping_rounds=5)

Training until validation scores don't improve for 5 rounds.
Iter   0, Train's L2: 0.3467775744, Valid's L2: 0.3438012841, Elapsed: 8.23 secs
Iter   1, Train's L2: 0.1910543084, Valid's L2: 0.2161812292, Elapsed: 8.05 secs
Training finished. Elapsed: 16.28 secs
 
*** Profile stats marshalled to file 'C:\\Users\\lenovo\\AppData\\Local\\Temp\\tmpmos3492_'.
Embedding SnakeViz in this document...
<function display at 0x0000019669E17C40>


<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Where is most of the time spent?</li>
         <li>What is the complexity of the <code>TreeNode.build</code> method as a function of the number of samples and the number of features?</li>
         <li>What could we do about this?</li>
     </ul>
</div>